### Ch07. 앙상블 학습과 랜덤 포레스트


**7.1 투표 기반 분류기**
- 직접 투표 분류기 : 각 분류기의 예측을 모아 가장 많이 선택된 클래스를 예측하는 것 (다수결 투표로 정해짐)
  - 약한 학습기: 랜덤 추측보다 조금 더 높은 성능을 내는 분류기
  - 강한 학습기: 높은 정확도를 내는 분류기

-> 즉 약한 학습기가 충분하게 많고 다양하다면 앙상블은 강한 학습기가 될 수 있음
  - 앙상블 방법은 예측기가 가능한 한 서로 독립적인 경우 최고의 성능을 발휘함
    - 분류기들이 연관성이 있는 경우 같은 종류의 오차를 만들기 쉽기 때문에 잘못된 클래스가 다수인 경우가 많고 앙상블의 정확도가 낮아지기 때문
- 여러 분류기를 조합하여 VotingClassifier라는 사이킷런의 투표기반 분류기를 형성하고 훈련하는 예시

In [1]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

# 5장에서의 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lf', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = 'hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lf', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [5]:
# 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.95
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


- 간접 투표(soft voting): 모든 분류기가 클래스의 확률을 예측할 수 있으면 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있음
  - voting = "soft"로 바꾸어 모든 분류기가 클래스의 확률을 추정할 수 있도록 함
  - probability = True로 지정 -> predict_proba()메서드를 사용 가능해짐
  

**7.2 배깅과 페이스팅**
- 배깅(bagging): 훈련 세트에서 중복을 허용하여 샘플링하는 방식
- 페이스팅(pasting): 중복을 허용하지 않고 샘플링하는 방식

-> 이 두가지는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있음
  - but 배깅만 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링 가능
- 모든 예측기에 훈련을 마치면, 앙상블은 모든 예측기의 예측을 모아 새로운 샘플에 대한 예측을 형성함
  - 이는 전형적으로 분류에선 통계적 최빈값을 이용하고, 회귀에선 평균을 계산함
- 개별 예측기는 훈련 세트로 훈련시킨 것에 비해 훨씬 크게 편향되어 있으나, 수집함수를 통과하며 편향과 분산이 모두 감소함
- 배깅과 페이스팅은 예측기가 모두 동시에 다른 CPU 코어나 서버에서 병렬로 학습시킬 수 있어 성능 향상에 효과적임

7.2.1 사이킷런의 배깅과 페이스팅
- BaggingClassifier를 이용
- 결정트리 분류기 500개의 앙상블을 훈련하는 예시

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

- 부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키기 때문에 배깅이 페이스팅보다 편향이 조금 더 높음
  - but 다양성을 추가하기 때문에 예측기 들의 상관관계가 줄어 앙상블의 분산이 감소됨

-> 따라서 CPU에 여유가 있다면 교차 검증을 통해 배깅과 페이스팅을 모두 평가해 좋은 성능을 보이는 것을 택하는게 좋음

7.2.2. oob 평가
- 배깅을 사용하면 어떤 샘플을 한 예측기를 위해 여러 번 샘플링되고, 어떤 것은 전혀 선택되지 않는 경우가 존재함
  - BaggingClassifier는 기본값으로 중복을 허용하기 때문
    - 이때 선택되지 않은 훈련 샘플을 oob 샘플이라고 부름 -> 이 샘플을 사용해 평가를 진행 가능
    
    -> 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻음
  - 사이킷런에서 BaggingClassifier의 oob_score = True로 설정하면 훈련이 끝난 후 자동으로 oob 평가를 수행함
- oob 평가를 자동으로 수행하는 코드 예시

In [8]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs =-1, oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.95

In [9]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

In [11]:
bag_clf.oob_decision_function_  # obb_decision_function_: oob 샘플에 대한 결정 함수 값을 얻는 코드

array([[0.        , 1.        ],
       [0.97765363, 0.02234637],
       [0.8       , 0.2       ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.72580645, 0.27419355],
       [1.        , 0.        ],
       [0.86263736, 0.13736264],
       [0.95789474, 0.04210526],
       [1.        , 0.        ],
       [0.9893617 , 0.0106383 ],
       [0.97916667, 0.02083333],
       [0.23115578, 0.76884422],
       [0.        , 1.        ],
       [0.00581395, 0.99418605],
       [0.06703911, 0.93296089],
       [1.        , 0.        ],
       [0.05454545, 0.94545455],
       [0.93406593, 0.06593407],
       [1.        , 0.        ],
       [0.1625    , 0.8375    ],
       [0.00529101, 0.99470899],
       [0.98255814, 0.01744186],
       [0.98378378, 0.01621622],
       [0.04477612, 0.95522388],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.93548387, 0.06451613],
       [0.89673913, 0.10326087],
       [0.76744186, 0.23255814],
       [0.

**7.3 랜덤 패치와 랜덤 서브스페이스**
- BaggingClassifier는 특성 샘플링도 지원함
  - 샘플링은 max_features, bootstrap_features의 2가지 매개변수로 조절됨
  - 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련됨
- 매우 고차원의 데이터셋을 다룰때 유용
- 랜덤 패치 방식: 훈련 특성과 샘플을 모두 샘플링하는 것
  - 랜덤 서브스페이스 방식: 훈련 샘플을 모두 사용하고, 특성은 샘플링하는 방법
- 특성 샘플링느 더욱 다양한 예측기를 만들어 편향을 늘리는 대신 분산을 낮추는 역할을 수행함

**7.4 랜덤 포레스트**
- 랜덤 포레스트: 일반적을 배깅(페이스팅)방법을 적용한 결정 트리의 앙상블
  - max_samples: 훈련 세트의 크기로 지정
  - RandomForestClassifier를 이용하여 사용
- 500개의 트리로 이뤄진 랜덤 포레스트 분류기를 여러 CPU 코어에서 훈련시키는 예시

In [12]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

- 랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더욱 주입함
  - 트리의 다양성 확보, 분산을 낮춤 -> 모델 성능 향상에 기여

In [13]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

7.4.1 엑스트라 트리
- 익스트림 랜덤 트리(엑스트라 트리): 극단적으로 무작위한 트리의 랜덤 포레스트
  - 편향은 늘어나나 분산을 낮추는 역할 수행
  - 일반적인 랜덤 포레스트보다 빠름
  - ExtraTreesClassifier를 통해 이용

7.4.2 특성 중요도
- 랜덤 포레스트는 특성의 상대적 중요도를 측정하기 쉽다는 장점이 존재
- 사이킷런은 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정함
- 사이킷런에선 훈련이 끝난 뒤 특성 별로 자동으로 중요도를 계산하고, 중요도의 전체 합이 1이 되도록 결과값을 정규화함
  - feature_importances_ 변수에 저장됨
- 랜덤 포레스트는 특성을 선택할 때 어떤 특성이 중요한지 빠르게 확인할 수 있어 매우 편리함

- iris 데이터셋을 이용하여 RandomForestClassifier을 훈련 후 각 특성의 중요도를 출력하는 예시

In [14]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09969245802761445
sepal width (cm) 0.024476630957221916
petal length (cm) 0.4210336239019445
petal width (cm) 0.45479728711321904


꽃잎의 길이와 너비가 중요하게 나타나고, 꽃받침의 길이와 너비는 비교적 덜 중요하게 나타남